In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor

dfTrain = pd.read_csv('../../../DataSet/trip_train.csv', parse_dates=['start_date', 'end_date'], infer_datetime_format=True)
dfTest = pd.read_csv('../../../DataSet/trip_test.csv', parse_dates=['start_date', 'end_date'], infer_datetime_format=True)
dfWeather = pd.read_csv('../../../DataSet/weather.csv', parse_dates=['date'], infer_datetime_format=True)
dfStation = pd.read_csv('../../../DataSet/station.csv')

dfWeather[(~dfWeather.precipitation_inches.isnull()) & (dfWeather.precipitation_inches.str.contains('T'))]

,date,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f,mean_dew_point_f,min_dew_point_f,max_humidity,mean_humidity,min_humidity,...,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,zip_code
32,2013-09-30,73.0,66.0,59.0,60.0,57.0,43.0,87.0,69.0,51.0,...,10.0,10.0,22.0,10.0,26.0,T,4.0,Fog,264.0,94107
59,2013-10-27,60.0,55.0,50.0,49.0,47.0,45.0,89.0,76.0,62.0,...,10.0,10.0,43.0,18.0,51.0,T,6.0,Rain,267.0,94107
75,2013-11-12,70.0,64.0,57.0,53.0,50.0,46.0,77.0,62.0,47.0,...,10.0,10.0,18.0,7.0,22.0,T,4.0,NaN,312.0,94107
84,2013-11-21,62.0,56.0,50.0,49.0,42.0,32.0,93.0,64.0,35.0,...,10.0,9.0,32.0,11.0,39.0,T,5.0,Rain,349.0,94107
95,2013-12-02,62.0,54.0,46.0,54.0,49.0,44.0,96.0,80.0,64.0,...,8.0,0.0,33.0,14.0,38.0,T,5.0,Fog-Rain,277.0,94107
131,2014-01-07,61.0,54.0,46.0,52.0,45.0,39.0,86.0,70.0,53.0,...,9.0,7.0,15.0,4.0,17.0,T,5.0,Rain,304.0,94107
133,2014-01-09,60.0,56.0,52.0,50.0,47.0,43.0,89.0,73.0,57.0,...,10.0,5.0,25.0,12.0,31.0,T,5.0,NaN,287.0,94107
135,2014-01-11,58.0,52.0,46.0,51.0,46.0,42.0,93.0,83.0,72.0,...,10.0,6.0,18.0,6.0,22.0,T,4.0,NaN,281.0,94107
153,2014-01-29,63.0,59.0,54.0,57.0,54.0,50.0,93.0,89.0,84.0,...,10.0,7.0,20.0,8.0,22.0,T,8.0,Rain,282.0,94107
154,2014-01-30,58.0,55.0,52.0,51.0,48.0,46.0,93.0,80.0,67.0,...,10.0,9.0,30.0,14.0,38.0,T,6.0,Rain,267.0,94107


In [2]:
dfTrain = dfTrain[['id', 'duration', 
                   'start_date', 'start_station_id', 
                   'subscription_type']]

print len(dfTrain)
dfTrain.head()

549961


,id,duration,start_date,start_station_id,subscription_type
0,907649,396,2015-08-27 08:36:00,50,Subscriber
1,384043,636,2014-07-28 22:06:00,67,Subscriber
2,316176,334,2014-06-09 08:42:00,77,Subscriber
3,618874,666,2015-01-26 16:55:00,69,Subscriber
4,910977,318,2015-08-29 15:09:00,67,Subscriber


In [3]:
dates = {'year':dfTrain.start_date.dt.year,
         'month':dfTrain.start_date.dt.month, 
         'day':dfTrain.start_date.dt.dayofyear, 
         'weekday':dfTrain.start_date.dt.dayofweek, 
         'hour':dfTrain.start_date.dt.hour}

wdates = {'year':dfWeather.date.dt.year, 
          'month':dfWeather.date.dt.month, 
          'day':dfWeather.date.dt.dayofyear}

dfTrain = dfTrain.join(pd.DataFrame(dates))
subscriptionTypes = dfTrain.subscription_type.unique()
dfTrain.subscription_type = dfTrain.subscription_type.astype('category', categories=subscriptionTypes).cat.codes

In [4]:
dfWeather = dfWeather.join(pd.DataFrame(wdates))[['year', 'month', 'day', 'zip_code',
                                                  'mean_temperature_f', 'mean_dew_point_f', 
                                                  'mean_humidity', 'mean_sea_level_pressure_inches', 
                                                  'mean_visibility_miles', 'mean_wind_speed_mph', 
                                                  'precipitation_inches']]
dfWeather.head()

,year,month,day,zip_code,mean_temperature_f,mean_dew_point_f,mean_humidity,mean_sea_level_pressure_inches,mean_visibility_miles,mean_wind_speed_mph,precipitation_inches
0,2013,8,241,94107,68.0,58.0,75.0,30.02,10.0,11.0,0
1,2013,8,242,94107,69.0,58.0,70.0,30.00,10.0,13.0,0
2,2013,8,243,94107,64.0,56.0,75.0,29.96,10.0,15.0,0
3,2013,9,244,94107,66.0,56.0,68.0,29.93,10.0,13.0,0
4,2013,9,245,94107,69.0,60.0,77.0,29.94,10.0,12.0,0


In [5]:
dfTrain = dfTrain[['id', 'duration', 'year', 
                   'month', 'weekday', 'day', 
                   'hour', 'start_station_id', 
                   'subscription_type']]

In [6]:
dfTrain.head()

,id,duration,year,month,weekday,day,hour,start_station_id,subscription_type
0,907649,396,2015,8,3,239,8,50,0
1,384043,636,2014,7,0,209,22,67,0
2,316176,334,2014,6,0,160,8,77,0
3,618874,666,2015,1,0,26,16,69,0
4,910977,318,2015,8,5,241,15,67,0


In [7]:
def cityNameToZipCode(row) :

    if row.city == 'San Francisco' :
        return 94107

    if row.city == 'Redwood City' :
        return 94063

    if row.city == 'Palo Alto' :
        return 94301

    if row.city == 'Mountain View' :
        return 94041

    if row.city == 'San Jose' :
        return 95113


In [8]:
dfStation.loc[:,'city'] = dfStation.apply(cityNameToZipCode, axis=1)
dfStation.rename(columns={'id':'start_station_id', 'city':'zip_code'}, inplace=True)
dfStation = dfStation[['start_station_id', 'zip_code']]



In [9]:
dfTrain = dfTrain.merge(dfStation, on='start_station_id')[['id', 'duration', 
                                                           'start_station_id', 'subscription_type', 
                                                           'weekday', 'year', 'month', 'day', 
                                                           'hour', 'zip_code']]

In [10]:
dfTrain = dfTrain.merge(dfWeather, on=['year', 'month', 'day', 'zip_code'])

In [11]:
print len(dfTrain)
dfTrain.head()

549961


,id,duration,start_station_id,subscription_type,weekday,year,month,day,hour,zip_code,mean_temperature_f,mean_dew_point_f,mean_humidity,mean_sea_level_pressure_inches,mean_visibility_miles,mean_wind_speed_mph,precipitation_inches
0,907649,396,50,0,3,2015,8,239,8,94107,72.0,55.0,57.0,30.01,10.0,6.0,0
1,907916,981,50,0,3,2015,8,239,9,94107,72.0,55.0,57.0,30.01,10.0,6.0,0
2,908620,585,50,0,3,2015,8,239,17,94107,72.0,55.0,57.0,30.01,10.0,6.0,0
3,907429,1772,50,0,3,2015,8,239,7,94107,72.0,55.0,57.0,30.01,10.0,6.0,0
4,907759,510,50,0,3,2015,8,239,8,94107,72.0,55.0,57.0,30.01,10.0,6.0,0


In [12]:
dfTrain.drop(['id','duration'], axis=1 , inplace=True)

In [13]:
def changeTforZero(row) :
    if row.precipitation_inches == 'T' :
        return 0
    
    return row.precipitation_inches

dfTrain = dfTrain.dropna()
dfTrain.precipitation_inches = dfTrain.apply(changeTforZero, axis=1)
dfTrain.precipitation_inches = pd.to_numeric(dfTrain.precipitation_inches)

In [15]:
dfTrain.isnull().any()

start_station_id                  False
subscription_type                 False
weekday                           False
year                              False
month                             False
day                               False
hour                              False
zip_code                          False
mean_temperature_f                False
mean_dew_point_f                  False
mean_humidity                     False
mean_sea_level_pressure_inches    False
mean_visibility_miles             False
mean_wind_speed_mph               False
precipitation_inches              False
dtype: bool